In [ ]:
#importing the libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import sqlite3
import warnings

# Suppress all warnings
warnings.filterwarnings('ignore')

In [ ]:
#loading the necessary dataset
#loading the tmdb movies csv file
tmdb=pd.read_csv("tmdb.movies.csv")
tmdb

In [ ]:
#creating a copy
tmdb1 = tmdb.copy(deep=True)
tmdb1

In [ ]:
#dropping unncessary columns
tmdb1.drop(["vote_average","vote_count","genre_ids","id"], axis=1,inplace=True)

In [ ]:
tmdb1

In [ ]:
#renaming the unnamed column'
tmdb1.rename(columns={"Unnamed":"id"},inplace=True)

In [ ]:
tmdb1

In [ ]:
tmdb1.rename(columns={"Unnamed: 0":"id"},inplace=True)

In [ ]:
tmdb1

In [ ]:
tmdb1["popularity"]=tmdb1["popularity"].round(1)

In [ ]:
tmdb1

In [ ]:
tmdb1["release_date"]= pd.to_datetime(tmdb1["release_date"]).dt.year

In [ ]:
tmdb1

In [ ]:
#checking for missing values 
tmdb1.isna().sum()

In [ ]:
#checking for duplicated values
tmdb1.duplicated().sum()

In [ ]:
bom=pd.read_csv("bom.movie_gross.csv")

In [ ]:
bom

In [ ]:
bom1 = bom.copy(deep=True)
bom1

In [ ]:
bom1.isna().sum()

In [ ]:
bom1.hist(bins=15,figsize=(6,6));

In [ ]:
bom1["domestic_gross"]=bom1["domestic_gross"].fillna(bom1["domestic_gross"].median())

In [ ]:
# Step 1: Remove commas from the 'foreign_gross' column
bom1['foreign_gross'] = bom1['foreign_gross'].replace({',': ''}, regex=True)

# Step 2: Convert the column to numeric values, using 'coerce' to handle errors
bom1['foreign_gross'] = pd.to_numeric(bom1['foreign_gross'], errors='coerce')

# Step 3: Replace NaN values with the median of the column
median_value = bom1['foreign_gross'].median()
bom1['foreign_gross'] = bom1['foreign_gross'].fillna(median_value)

In [ ]:
bom1

In [ ]:
#dropping unnecessary columns
bom1.drop(['studio'],axis=1,inplace=True)

In [ ]:
bom1.isnull().sum().any()

In [ ]:
bom1

In [ ]:
#adding a new column
bom1["total_gross"]=bom1["domestic_gross"]+bom1["foreign_gross"]
bom1

In [ ]:
#changing to currency for uniformity 
bom1['domestic_gross'] = bom1['domestic_gross'].apply(lambda x: f"${x:,.2f}")

In [ ]:
bom1['foreign_gross'] = bom1['foreign_gross'].apply(lambda x: f"${x:,.2f}")

In [ ]:
bom1['total_gross'] = bom1['total_gross'].apply(lambda x: f"${x:,.2f}")

In [ ]:
bom1

In [ ]:
#confirming the changes
bom1

In [ ]:
#adding the id as the  first column
bom1.insert(0, 'id', range(1, len(bom1) + 1))


In [ ]:
bom1

In [ ]:
#loading the sql database
mvsql=sqlite3.connect("im.db")

In [ ]:
#viewing the tables
pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", mvsql)

In [ ]:
#loading the movie basics table from the sql database
mvbas=pd.read_sql("""SELECT * FROM movie_basics;""",mvsql)

In [ ]:
mvbas

In [ ]:
#creating a new copy
mvbas1 = mvbas.copy(deep=True)
mvbas1

In [ ]:
#dropping unnecesary columns
mvbas1.drop(["original_title","runtime_minutes"],axis=1,inplace=True)

In [ ]:
mvbas1

In [ ]:
#renaming columns for uniformity
mvbas1.rename(columns={"movie_id":'id'},inplace=True)

In [ ]:
mvbas1.rename(columns={"start_year":'year'},inplace=True)

In [ ]:
mvbas1.rename(columns={"primary_title":'title'},inplace=True)

In [ ]:
mvbas1

In [ ]:
#loading the movie rating table
mvrat=pd.read_sql("""SELECT * FROM movie_ratings;""",mvsql)

In [ ]:
#creating a new copy
mvrat1= mvrat.copy(deep=True)

In [ ]:
mvrat1

In [ ]:
#dropping unnecessary columns
mvrat1.drop(["numvotes"],axis=1,inplace=True)

In [ ]:
#renaming columns
mvrat1.rename(columns={"movie_id":"id"},inplace=True)

In [ ]:
mvrat1.rename(columns={"averagerating":"average_rating"},inplace=True)

In [ ]:
mvrat1

In [ ]:
# Merge tables from the sql which are relatabe specifying same column names (id)
merged_movies = pd.merge(mvbas1, mvrat1, left_on='id', right_on='id', how='inner')


print(merged_movies)

In [ ]:
#changing into a dataframe
merged_movies_df=pd.DataFrame(merged_movies)
merged_movies_df

In [ ]:
#Merging the tm1 dataset("tmdb.movies.csv") to the original merged data set
# First merge: Add tmdb1 to the existing merged DataFrame
merged_movies_final = pd.merge(merged_movies, tmdb1, on="title", how='inner')

# Second merge: Add bom1 to the updated merged DataFrame using the same column name
merged_movies_final = pd.merge(merged_movies, bom1, on='title', how='inner')

# Check the result
print(merged_movies_final.head())


In [ ]:
#loading the dataframe
merged_movies_final_df=pd.DataFrame(merged_movies_final)
merged_movies_final_df

In [ ]:
#dropping unecessary/repeated columns for the merged datasets
merged_movies_final_df.drop(columns=["id_x","year_x","id_y"],axis=1,inplace=True)

In [ ]:
merged_movies_final_df

In [ ]:
#checking for missing values
merged_movies_final_df.isna().sum()

In [ ]:
#filling the missing values
for column in merged_movies_final_df.select_dtypes(include=["object"]).columns:
    merged_movies_final_df[column].fillna(merged_movies_final_df[column].mode()[0],inplace=True)

In [ ]:
#confirming changes
merged_movies_final_df.isna().sum()

In [ ]:
#renaming columns
merged_movies_final_df.rename({"year_y":"year"},axis=1,inplace=True)

In [ ]:
merged_movies_final_df

In [ ]:
#checking for outliers
sns.boxplot(merged_movies_final_df)

In [ ]:
#checking if all the necessary data is available
merged_movies_final_df.info()

In [ ]:
#loading the second dataset
rt = pd.read_csv("rt.movie_info.tsv",delimiter="\t")
rt

In [ ]:
#creating a copy
rt1 =rt.copy(deep=True)
rt1

In [ ]:
#dropping unecessary columns
rt1.drop(["synopsis","writer","dvd_date","currency","runtime"],axis=1,inplace=True)

In [ ]:
rt1

In [ ]:
#checking for missing values
rt1.isna().sum()

In [ ]:
#filling missing values for objects using mode
for column in rt1.select_dtypes(include=["object"]).columns:
    rt1[column].fillna(rt1[column].mode()[0],inplace=True)

In [ ]:
#changing the box office values to numerical(box office sales a movie makes after its release)
#noted it was not numerical since it gave out an error when trying to fill missing values
# Step 1: Remove commas
rt1['box_office'] = rt1['box_office'].str.replace(',', '')

# Step 2: Convert to integers
rt1['box_office'] = rt1['box_office'].astype(int)



In [ ]:
#checking for uniformity using a histogram
rt1.hist(bins=15,figsize=(5,5));

In [ ]:
#filling in the missing values
rt1["box_office"]=rt1["box_office"].fillna(rt1["box_office"].median())

In [ ]:
#confirming no missing values
rt1.isnull().sum().any()

In [ ]:
rt1

In [ ]:
#converting the date time to year only for easier analysis
rt1["theater_date"]= pd.to_datetime(rt1["theater_date"]).dt.year

In [ ]:
# Split 'genre' into 'main_genre' and 'supporting_genre', handling NaN values
rt1['main_genre'] = rt1['genre'].str.split('|').str[0]
rt1['supporting_genre'] = rt1['genre'].str.split('|').apply(
    lambda x: '|'.join(x[1:]) if isinstance(x, list) and len(x) > 1 else ''
)


# Preview the result
rt1[['genre', 'main_genre', 'supporting_genre']].head()



In [ ]:
rt1

In [ ]:
#dropping the unnecessary new column created
rt1.drop(["genre","supporting_genre"],axis=1,inplace=True)

In [ ]:
rt1.rename({"main_genre":"genre"},axis=1,inplace=True)

In [ ]:
rt1

In [ ]:
merged_movies_final_df.to_csv("merged movies.csv")

In [ ]:
rt1.to_csv("movie ratings.csv")

In [ ]:
merged_movies_final_df.head()

In [ ]:
rt1.head()

In [ ]:


# Genre vs Rating 
genre_rating= merged_movies_final_df.groupby("genres")["average_rating"].sum().sort_values(ascending=False).head(10)
plt.figure(figsize=(5, 5))
sns.barplot(x=genre_rating.index, y=genre_rating.values,palette="Blues")
plt.xticks(rotation=45, ha='right')
plt.title('Genre vs Rating')
plt.show()


In [ ]:
#Genre vs Sales
genre_sales= merged_movies_final_df.groupby("genres")["total_gross"].sum().sort_values(ascending=False).head(10)
plt.figure(figsize=(5, 5))
sns.barplot(x=genre_sales.index, y=genre_rating.values,palette="Blues")
plt.xticks(rotation=45, ha='right')
plt.title('Genre vs Sales')
plt.show()

In [ ]:
#popular genres based on their count
merged_movies_final_df["genres"].value_counts().head(5).plot(kind='bar',figsize=(5,5))
sns.set_palette('Blues')
plt.title("popular genres")
plt.ylabel("counts")
plt.xlabel('genre')
plt.xticks(rotation=45);

In [ ]:
#top movies based on genre popularity
top_three_movies = merged_movies_final_df["title"].value_counts().head(3).index

filtered_data = merged_movies_final_df[merged_movies_final_df["title"].isin(top_three_movies)]

top_genres=(
    filtered_data.groupby("title")["genres"]
    .value_counts()
    .groupby(level=0).nlargest(3)
    .reset_index(level=0,drop=True)
    .index.get_level_values(1)
)
    
filtered_data=filtered_data[filtered_data['genres'].isin(top_genres)]

plt.figure(figsize=(5, 5))#select figure size
sns.countplot(data=filtered_data, x='title', hue='genres')#selecting type of graph(countplot)
plt.title('Top 3 Movies with Top 3 Genres Vs popularity')#title for the graph
plt.xlabel('Movie')#x-axis label
plt.ylabel('Popularity')#y-axis label
plt.legend(title='Genres')#legend title
plt.legend(fontsize='small')  # You can use 'small', 'medium', 'large' or specify a number like 8 or 10

# Alternatively, you can control the size of the legend box:
plt.legend(handlelength=1, fontsize=8)  #
plt.show()#visualize the graph



In [ ]:
#how genre with best rating has faired over the years(univariate analysis)
# Filter for the genre of interest
genre_of_interest = "Drama"
drama_df = merged_movies_final_df[merged_movies_final_df['genres'].str.contains(genre_of_interest, na=False)]

# Perform univariate analysis: Focus on average ratings over time
drama_yearly = drama_df.groupby('year')['average_rating'].mean()

# Plotting the univariate trend
plt.figure(figsize=(5, 5))
drama_yearly.plot(kind='line', marker='o', color='blue')
plt.title(f"Univariate Analysis: Drama Genre Ratings Over the Years", fontsize=16)
plt.xlabel("Year", fontsize=12)
plt.ylabel("Average Rating", fontsize=12)
plt.grid(visible=True, linestyle='--', alpha=0.6)
plt.show()

In [ ]:
#popular directors
director_popularity = rt1['director'].value_counts().head(5)
plt.figure(figsize=(5,5))
sns.barplot(x=director_popularity.index,y=director_popularity.values)
plt.title("most popular directors")
plt.xlabel("director")
plt.ylabel("count")
plt.xticks(rotation=45);

In [ ]:
#best selling directors

# Group by director and sum the box office sales
director_sales = rt1.groupby("director")["box_office"].sum().sort_values(ascending=False).head(10)

# Create the plot
plt.figure(figsize=(5, 5))
sns.barplot(x=director_sales.values, y=director_sales.index, palette="Blues", orient="h")  # Horizontal bars

# Add title and labels
plt.xlabel('Box Office Sales')
plt.title('Directors vs Sales')

# Include x-tick labels (rotate for readability)
plt.xticks(rotation=60)

# Show the plot
plt.show()

In [ ]:
#popular movie rating
rating_popularity = rt1['rating'].value_counts().head(5)
plt.figure(figsize=(5,5))
sns.barplot(x=rating_popularity.index,y=rating_popularity.values)
plt.title("most popular movie rating")
plt.xlabel("movie rating")
plt.ylabel("count")
plt.xticks(rotation=45);

In [ ]:
#univariate analysis based on the highest selling director
# Filter for the genre of interest
highest_director = "Steven Spielberg"
director_df = rt1[rt1['director'].str.contains(highest_director, na=False)]

# Perform univariate analysis: Focus on average ratings over time
performance_yearly = director_df.groupby('theater_date')['box_office'].mean()

# Plotting the univariate trend
plt.figure(figsize=(5, 5))
drama_yearly.plot(kind='line', marker='o', color='blue')
plt.title(f"Univariate Analysis: Steven Spielberg's sales Over the Years", fontsize=16)
plt.xlabel("Year", fontsize=12)
plt.ylabel("Sales", fontsize=12)
plt.grid(visible=True, linestyle='--', alpha=0.6)
plt.show()